In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import *
from torch import nn
from torch.utils.data.sampler import SubsetRandomSampler

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from pathlib import Path
import os
import random

path = Path("/content/drive/My Drive/SIIM-ISIC")

## Carga y procesamiento de los datos

### Procesamiento
En esta sección se cargan los datos a partir del archivo train_concat.csv, obtenido desde ...

Este conjunto de datos incluye más imágenes de melanomas que el original de la competición y sus imágenes tienen como resolución máxima 256x256. En total son alrededor de 37000, de los cuales 5000 son imágenes de melanomas (frente a las 500 muestras originales).

Debido a las limitaciones de Google Drive se ha reducido el tamaño del dataset ejecutando la siguiente línea de código, dejando el dataset en 12000 imágenes, siendo 5000 positivos (melanomas) y 7000 negativos.

In [ ]:
# Esta sección está comentada para evitar ejecuciones no voluntarias
"""

d = pd.read_csv(path/"train_concat.csv")
positive = d[d.target==1]
neg_idx = list(set(list(range(len(d)))) - set(positive.index))
negative = d.iloc[neg_idx]
new_data = positive.append(negative.sample(n=7000), ignore_index=True).sample(frac=1).reset_index(drop=True)
new_data.to_csv(path/"short_data.csv", index=False)

"""

'\n\nd = pd.read_csv(path/"train_concat.csv")\npositive = d[d.target==1]\nneg_idx = list(set(list(range(len(d)))) - set(positive.index))\nnegative = d.iloc[neg_idx]\nnew_data = positive.append(negative.sample(n=7000), ignore_index=True).sample(frac=1).reset_index(drop=True)\nnew_data.to_csv(path/"short_data.csv", index=False)\n\n'

### Carga

In [ ]:
def split_from_csv(path_csv, pct_split):
    total_data = pd.read_csv(path_csv)
    valid_data = total_data.sample(frac=pct_split)
    idxs = list(set(list(range(len(total_data)))) - set(valid_data.index))
    train_data = total_data.iloc[idxs]

    return train_data.reset_index(drop=True), valid_data.reset_index(drop=True)

In [ ]:
t, v = split_from_csv(path/"short_data.csv", 0.2)
len(t), len(v)

(9685, 2421)

#### Dataset sin metadatos
Debe usarse para el modelo de la ResNet modificada y para el primer modelo con EfficientNet.

In [ ]:
class Dataset_from_csv(Dataset):
    """
    Dataset creado teniendo las imágenes en una carpeta y las etiquetas en un 
    csv en el que se mencionan los nombres de las imágenes de entrada a las que 
    corresponde cada etiqueta.
    """
    def __init__(self, path, dataframe, dir_images, transforms=None):
        """
        Args:
            path (Path Object): Ruta del directorio principal
            csv_file (string): Nombre del csv.
            dir_images (string): Ruta a la carpeta con las imágenes.
            transforms (callable, optional): Lista de posibles transformaciones
                hecha con Compose. Debe acabar con ToTensor.
        """
        self.path = path
        self.labels = dataframe
        self.path_images = dir_images
        self.transforms = transforms

        #Cargamos las imágenes para facilitar la velocidad del entrenamiento
        self.loaded_images = {}
        count = 0
        print("Inicio de la carga de archivos.")
        for name in dataframe.image_name:
            if (count%20)==0: print(f"Progreso: {round(count*100/len(dataframe), 2)}%")
            img_name = path/dir_images/f"{name}.jpg"
            image = cv2.imread(os.path.join(img_name))
            self.loaded_images[name] = image
            count += 1

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        img_name = self.labels.iloc[idx, 0]
        image = self.loaded_images[img_name]
        im_label = np.array(self.labels.iloc[idx].target.squeeze(-1)).astype("float") # reshape(-1,2) si son varias etiquetas
        sample = {"image": image, "label": im_label}

        if self.transforms:
            sample["image"] = self.transforms(sample["image"])

        return sample

#### Dataset con metadatos
Debe usarse para el modelo de EfficientNet con metadatos.

Los datos incluyen más información que la imagen y el diagnóstico. Alguno de estos datos accesorios son el sexo, la edad y el lugar anatómico en el que se ha detectado la mancha.

In [ ]:
class ImageDataset_label_from_dataframe(Dataset):
    """
    Dataset creado teniendo las imágenes en una carpeta y las etiquetas en un 
    csv en el que se mencionan los nombres de las imágenes de entrada a las que 
    corresponde cada etiqueta.
    """
    def __init__(self, path, dataframe, dir_images, transforms=None):
        """
        Args:
            path (Path Object): Ruta del directorio principal
            csv_file (string): Nombre del csv.
            dir_images (string): Ruta a la carpeta con las imágenes.
            transforms (callable, optional): Lista de posibles transformaciones
                hecha con Compose. Debe acabar con ToTensor.
        """
        self.path = path
        self.data = dataframe
        self.meta = pd.concat([pd.get_dummies(dataframe[["sex", "anatom_site_general_challenge"]]), dataframe["age_approx"].fillna(dataframe.age_approx.mean())], axis=1)
        self.path_images = dir_images
        self.transforms = transforms

        #Cargamos las imágenes para facilitar la velocidad del entrenamiento
        self.loaded_images = {}
        count = 0
        print("Inicio de la carga de archivos.")
        for name in self.data.image_name:
            if (count%20)==0: print(f"Progreso: {round(count*100/len(dataframe), 2)}%")
            img_name = path/dir_images/f"{name}.jpg"
            image = cv2.imread(os.path.join(img_name))
            self.loaded_images[name] = image
            count += 1

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_name = self.data.iloc[idx, 0]
        image = self.loaded_images[img_name]
        im_label = np.array(self.data.iloc[idx].target.squeeze(-1)).astype("float") # reshape(-1,2) si son varias etiquetas
        sample = {"image": image, "label": im_label}
        sample_meta = np.array(self.meta.iloc[idx].values, dtype=np.float32)

        if self.transforms:
            sample["image"] = self.transforms(sample["image"])

        return sample, sample_meta

#### Data augmentation y carga de los datos en RAM

Las imágenes de manchas en la piel permiten un *data augmentation* bastante agresivo, combinando volteos de la imagen (tanto en el eje vertical como en el horizontal) y rotaciones. 

Debido a que el *dataset* ampliado incluye imágenes de melanomas vistas a través de un microscopio incluímos una transformación que simula este efecto para que estar rodeado de una zona negra no permita a la red concluir que la imagen es un melanoma.

In [ ]:
# Tomado de https://www.kaggle.com/c/siim-isic-melanoma-classification/discussion/159476
import cv2
class Microscope:
    def __init__(self, p: float = 0.5):
        self.p = p

    def __call__(self, img):
        if random.random() < self.p:
            circle = cv2.circle((np.ones(img.shape) * 255).astype(np.uint8),
                        (img.shape[0]//2, img.shape[1]//2),
                        random.randint(img.shape[0]//2 - 3, img.shape[0]//2 + 15),
                        (0, 0, 0),
                        -1)

            mask = circle - 255
            img = np.multiply(img, mask)

        return img

    def __repr__(self):
        return f'{self.__class__.__name__}(p={self.p})'

In [ ]:
size = 128
!pip install torchtoolbox
# No se usan las transformaciones de Pytorch porque hemos abierto las imágenes con cv2
import torchtoolbox.transform as trans

tsfm = trans.Compose([
        trans.Resize(size),
        Microscope(),
        trans.RandomHorizontalFlip(),
        trans.RandomVerticalFlip(),
        trans.RandomRotation(30),
        trans.ToTensor(),
        #[0.5568, 0.5837, 0.7436] mean
        #[0.1409, 0.1247, 0.1067] std
        trans.Normalize((0.5568, 0.5837, 0.7436), (0.1409, 0.1247, 0.1067))
    ])

tsfm2 = trans.Compose([
        trans.Resize(size),
        trans.ToTensor(),
        trans.Normalize((0.5568, 0.5837, 0.7436), (0.1409, 0.1247, 0.1067))
    ])

     |████████████████████████████████| 61kB 5.5MB/s 


Cargamos los datos en RAM porque Google Drive toma mucho tiempo para abrir las imágenes. Es por esto que abrimos todas las imágenes al principio para tenerlos ya en memoria, concretamente en una lista, para agilizar el acceso.

In [ ]:
folder = "train"
train_dataset = Dataset_from_csv(path, t, folder, tsfm)
valid_dataset = Dataset_from_csv(path, v, folder, tsfm2)

Inicio de la carga de archivos.
Progreso: 0.0%
Progreso: 0.21%
Progreso: 0.41%
Progreso: 0.62%
Progreso: 0.83%
Progreso: 1.03%
Progreso: 1.24%
Progreso: 1.45%
Progreso: 1.65%
Progreso: 1.86%
Progreso: 2.07%
Progreso: 2.27%
Progreso: 2.48%
Progreso: 2.68%
Progreso: 2.89%
Progreso: 3.1%
Progreso: 3.3%
Progreso: 3.51%
Progreso: 3.72%
Progreso: 3.92%
Progreso: 4.13%
Progreso: 4.34%
Progreso: 4.54%
Progreso: 4.75%
Progreso: 4.96%
Progreso: 5.16%
Progreso: 5.37%
Progreso: 5.58%
Progreso: 5.78%
Progreso: 5.99%
Progreso: 6.2%
Progreso: 6.4%
Progreso: 6.61%
Progreso: 6.81%
Progreso: 7.02%
Progreso: 7.23%
Progreso: 7.43%
Progreso: 7.64%
Progreso: 7.85%
Progreso: 8.05%
Progreso: 8.26%
Progreso: 8.47%
Progreso: 8.67%
Progreso: 8.88%
Progreso: 9.09%
Progreso: 9.29%
Progreso: 9.5%
Progreso: 9.71%
Progreso: 9.91%
Progreso: 10.12%
Progreso: 10.33%
Progreso: 10.53%
Progreso: 10.74%
Progreso: 10.94%
Progreso: 11.15%
Progreso: 11.36%
Progreso: 11.56%
Progreso: 11.77%
Progreso: 11.98%
Progreso: 12.18%
Pro

## Modelos 

Para entrenarlos se seguirá el siguiente proceso:


1.   Carga de los datos en un formato de 128x128
2.   Entrenamiento a aproximadamente una razón de aprendizaje de 0'0003.
3.   Una vez el aprendizaje se ha ralentizado cambiar los datos al formato de 256x256.
4.   Continuar entrenando unas pocas iteracciones a 0'0003 y bajar su valor en cuanto se ralentice a 0'00003.

Se ha probado el uso de la política de entrenamiento *One Cycle* y no parece dar un resultado mejor que el de un entrenamiento normal. En el caso de que se quiera usar el *Scheduler* están las líneas necesarias comentandas dentro de la clase *Learner*.

El umbral que aparece como parámetro de la función *fit* de la clase *Learner* indica a partir de qué nivel de activación en la neurona de salida se considerará que es un melanoma. En este caso parece que el mejor número para este problema es 0.5, pero podría variar si empleamos esta arquitectura para resolver otro problema (como una clasificación multietiqueta o una clasificación con una clase "desconocido", como es este caso).



### Resnet modificada

Resblock modificado según el paper *Bag of Tricks for Image Classification with Convolutional Neural Networks* (disponible en arxiv.org/pdf/1812.01187.pdf)

In [ ]:
def noop(x): return x # Función identidad

def conv(ni, nf, ks=3, stride=1, bias=False): #nn.Conv2d simplificada con padding de tamaño dependiente del tamaño de filtro
    return nn.Conv2d(ni, nf, kernel_size=ks, stride=stride, padding=ks//2, bias=bias)

act_fn = nn.ReLU(inplace=True)

def conv_layer(ni, nf, ks=3, stride=1, zero_bn=False, activacion=True):
    """
    Bloque Conv+BN+ReLU(opcional)
    BN con gamma inicializada a cero de forma opcional
    """
    bn = nn.BatchNorm2d(nf)
    nn.init.constant_(bn.weight, 0. if zero_bn else 1.) # Truco para anular la mitad que procesa del ResBlock y facilitar el entrenamiento inicial.
    layers = [conv(ni, nf, ks, stride=stride), bn]
    if activacion: layers.append(act_fn) # Optativo porque en la última convolución del ResBlock la activación va tras la suma de los dos caminos.
    return nn.Sequential(*layers)

In [ ]:
class Resblock(nn.Module):
    def __init__(self, ni, nh, stride=1):
        super().__init__()
        # Camino A
        layers = [conv_layer(ni, nh, 3, stride=stride), conv_layer(nh, nh, 3, zero_bn=True, activacion=False)]
        self.convs = nn.Sequential(*layers)
        # Camino B
        self.idconv = noop if ni==nh else conv_layer(ni, nh, 1, activacion=False) # Si difiere el número de canales
        self.pool = noop if stride==1 else nn.AvgPool2d(2, ceil_mode=True)        # Si difiere el tamaño de los datos
    
    def forward(self, x):
        return act_fn(self.convs(x) + self.idconv(self.pool(x)))

In [ ]:
def group(ni, nf, n_blocks, stride):
    """
    Crea un conjunto de ResBlocks y los devuelve.
    Solo se usará stride=2 en el primer bloque del grupo
    Tendrán el mismo número de filtros de salida que de entrada excepto en
    el primer bloque del grupo.
    """
    return nn.Sequential(
        *[Resblock(ni if i==0 else nf, nf, stride if i==0 else 1) for i in range(n_blocks)]
        )

In [ ]:
def init_weights(m):
    if getattr(m, 'bias', None) is not None: nn.init.constant_(m.bias, 0)
    if isinstance(m, (nn.Conv2d,nn.Linear)): nn.init.kaiming_normal_(m.weight)
    for l in m.children(): init_weights(l) # Recursivo

In [ ]:
class XResNet18(nn.Sequential):
    @classmethod
    def create(cls, c_in=3, c_out=1):
        list_nf = [c_in, (c_in+1)*8, 64, 64]
        inicio = [conv_layer(list_nf[i], list_nf[i+1], 3, stride=2 if i==0 else 1) for i in range(3)]
        list_nf = [64, 64, 128, 256, 512]
        layers = [2, 2, 2, 2]
        b_groups = [group(list_nf[i], list_nf[i+1], n_blocks=1, stride=1 if i==0 else 2) for i,l in enumerate(layers)]
        
        resnet = cls(*inicio,
                     nn.MaxPool2d(kernel_size=3, stride=2, padding=1),
                     *b_groups,
                     nn.AdaptiveAvgPool2d(1), # Media entre canales
                     nn.Flatten(),
                     nn.Linear(list_nf[-1], c_out)
                     )
        
        init_weights(resnet)
        return resnet

In [ ]:
model = XResNet18.create()
model.cuda()

In [ ]:
model

XResNet18(
  (0): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (1): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (2): Sequential(
    (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (4): Sequential(
    (0): Resblock(
      (convs): Sequential(
        (0): Sequential(
          (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (1): BatchNorm2d(64, eps=1e-05, mo

In [ ]:
def accuracy(y_pred, y):
    n_true = torch.sum(y_pred.data.max(1, keepdim=True)[1].squeeze(-1) == y).item()
    n_samples = y.size()[0]
    return n_true, n_samples

def multilabel_accuracy(y_pred, y, threshold):
    n_true, n_samples = 0, len(y_pred)
    for pred,target in zip(y_pred, y):
        if len(pred[pred>threshold])>1:
            pred_label = (pred==(pred[pred>threshold].max())).nonzero()
        else:
            pred_label = ((pred>threshold)==True).nonzero()
        label = (target==True).nonzero()
        if str(pred_label)==str(label): n_true += 1
    return n_true, n_samples

class Learner():
    def __init__(self, model, train, valid, optim, loss_fn):
        self.model = model
        self.train = train
        self.valid = valid
        self.optim = optim
        self.loss_fn = loss_fn
        self.lr = 0
        self.losses_train = []
        self.losses_valid = []

    def fit(self, epochs, lr, max_lr, threshold):
        #scheduler = torch.optim.lr_scheduler.OneCycleLR(self.optim, max_lr=max_lr, steps_per_epoch=len(self.train), epochs=epochs)
        if lr != self.lr:
            self.lr == lr
            for param_group in optimizer.param_groups:
                param_group['lr'] = lr
        
        for epoch in range(epochs):
            acc_loss = 0
            n_true = 0
            n_samples = 0
            acc = 0
            
            acc_loss_val = 0
            n_true_val = 0
            n_samples_val = 0

            # Training
            self.model = self.model.train()
            for count, sample in enumerate(self.train):
                X_batch = sample["image"].to("cuda:0")
                y_batch = sample["label"].float().to("cuda:0")
                y_pred = self.model(X_batch)
                loss = self.loss_fn(y_pred.squeeze(-1), y_batch)
                n_true_b, n_samples_b = multilabel_accuracy(y_pred.squeeze(-1), y_batch, threshold) 
                n_true += n_true_b
                n_samples += n_samples_b
                if count % 50 == 0:
                    acc = n_true/n_samples
                    print(f"Train epoch {epoch+1}: [{count}/{len(self.train)}\tLoss: {round(loss.item(), 3)}\tAcc: {round(acc*100, 2)}%]")
                acc_loss += loss.item()

                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                #scheduler.step()

            # Validation
            self.model = self.model.eval()
            i=0
            with torch.no_grad():
                for sample_val in self.valid:
                    X_batch = sample_val["image"].to("cuda:0")
                    y_batch = sample_val["label"].float().to("cuda:0")
                    y_pred = self.model(X_batch)
                    loss = self.loss_fn(y_pred.squeeze(-1), y_batch)
                    n_true_b, n_samples_b = multilabel_accuracy(y_pred.squeeze(-1), y_batch, threshold) 
                    n_true_val += n_true_b
                    n_samples_val += n_samples_b
                    acc_loss_val += loss.item()

            acc = n_true/n_samples
            acc_val = n_true_val/n_samples_val
            
            print(f"Train epoch {epoch+1}: Train -> [Loss: {round(acc_loss/len(self.train), 3)}\tAcc: {round(acc*100, 2)}%], Valid -> [Loss: {round(acc_loss_val/len(self.valid), 3)}\tAcc: {round(acc_val*100, 2)}%]\n")
            self.losses_train.append(acc_loss/len(self.train))
            self.losses_valid.append(acc_loss_val/len(self.valid))

    def plot():
        plt.plot(learner.losses_train)
        plt.plot(learner.losses_valid)
        plt.ylabel('Error')
        plt.xlabel('Epochs')
        plt.legend(["Entrenamiento", "Validación"])
        plt.show()

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_fn = torch.nn.BCEWithLogitsLoss()

Se ajusta de forma automáticamente el tamaño del lote en función del tamaño de las imágenes para que la RAM de la GPU no colapse, pues eso nos obligaría a comenzar de nuevo con la ejecución tras reiniciar el entorno de ejecución.

In [ ]:
if size == 128: 
    bs=256
else: 
    bs = 64

train_dataloader = DataLoader(train_dataset, batch_size=bs, shuffle=True, num_workers=0)
valid_dataloader = DataLoader(valid_dataset, batch_size=bs, shuffle=True, num_workers=0)

In [ ]:
learner = Learner(model, train_dataloader, valid_dataloader, optimizer, loss_fn)

En ocasiones, al ejecutar la celda de abajo aparece un error relacionado con "ndim". Sin conocer el motivo detrás de él, se soluciona volviendo a cargar los datos en RAM.

In [ ]:
learner.train=train_dataloader
learner.valid=valid_dataloader
learner.fit(3, 0.0003, 0.1, 0.5)

In [ ]:
learner.plot()

### EfficientNet con Transfer Learning

Esta arquitectura se descarga con unos pesos ya entrenados con Imagenet, lo que facilita el aprendizaje.

In [ ]:
!pip install efficientnet_pytorch
from efficientnet_pytorch import EfficientNet

def NewEfficientNet(c_out=1):
    model = EfficientNet.from_pretrained('efficientnet-b1')
    model._fc = nn.Linear(in_features=1280, out_features=c_out, bias=True)
    return model

model = NewEfficientNet()
model = model.cuda()

  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.0-cp36-none-any.whl size=16031 sha256=651fbb4f40c660972d1d689b599746ff002c58d77f4008534df6efdef188e33c
  Stored in directory: /root/.cache/pip/wheels/e9/c6/e1/7a808b26406239712cfce4b5ceeb67d9513ae32aa4b31445c6
Successfully built efficientnet-pytorch


Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b1-f1951068.pth" to /root/.cache/torch/hub/checkpoints/efficientnet-b1-f1951068.pth



Loaded pretrained weights for efficientnet-b1


In [ ]:
model

EfficientNet(
  (_conv_stem): Conv2dStaticSamePadding(
    3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False
    (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
  )
  (_bn0): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
  (_blocks): ModuleList(
    (0): MBConvBlock(
      (_depthwise_conv): Conv2dStaticSamePadding(
        32, 32, kernel_size=(3, 3), stride=[1, 1], groups=32, bias=False
        (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
      )
      (_bn1): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
      (_se_reduce): Conv2dStaticSamePadding(
        32, 8, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_se_expand): Conv2dStaticSamePadding(
        8, 32, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_project_conv): Conv2dStaticSamePadding(
        32, 16, kernel_size=

In [ ]:
def accuracy(y_pred, y):
    n_true = torch.sum(y_pred.data.max(1, keepdim=True)[1].squeeze(-1) == y).item()
    n_samples = y.size()[0]
    return n_true, n_samples

def multilabel_accuracy(y_pred, y, threshold):
    n_true, n_samples = 0, len(y_pred)
    for pred,target in zip(y_pred, y):
        if len(pred[pred>threshold])>1:
            pred_label = (pred==(pred[pred>threshold].max())).nonzero()
        else:
            pred_label = ((pred>threshold)==True).nonzero()
        label = (target==True).nonzero()
        if str(pred_label)==str(label): n_true += 1
    return n_true, n_samples

class Learner():
    def __init__(self, model, train, valid, optim, loss_fn):
        self.model = model
        self.train = train
        self.valid = valid
        self.optim = optim
        self.loss_fn = loss_fn
        self.lr = 0
        self.losses_train = []
        self.losses_valid = []

    def fit(self, epochs, lr, max_lr, threshold):
        #scheduler = torch.optim.lr_scheduler.OneCycleLR(self.optim, max_lr=max_lr, steps_per_epoch=len(self.train), epochs=epochs)
        if lr != self.lr:
            self.lr == lr
            for param_group in optimizer.param_groups:
                param_group['lr'] = lr
        
        for epoch in range(epochs):
            acc_loss = 0
            n_true = 0
            n_samples = 0
            acc = 0
            
            acc_loss_val = 0
            n_true_val = 0
            n_samples_val = 0

            # Training
            self.model = self.model.train()
            for count, sample in enumerate(self.train):
                X_batch = sample["image"].to("cuda:0")
                y_batch = sample["label"].float().to("cuda:0")
                y_pred = self.model(X_batch)
                loss = self.loss_fn(y_pred.squeeze(-1), y_batch)
                n_true_b, n_samples_b = multilabel_accuracy(y_pred.squeeze(-1), y_batch, threshold) 
                n_true += n_true_b
                n_samples += n_samples_b
                if count % 50 == 0:
                    acc = n_true/n_samples
                    print(f"Train epoch {epoch+1}: [{count}/{len(self.train)}\tLoss: {round(loss.item(), 3)}\tAcc: {round(acc*100, 2)}%]")
                acc_loss += loss.item()

                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                #scheduler.step()

            # Validation
            self.model = self.model.eval()
            i=0
            with torch.no_grad():
                for sample_val in self.valid:
                    X_batch = sample_val["image"].to("cuda:0")
                    y_batch = sample_val["label"].float().to("cuda:0")
                    y_pred = self.model(X_batch)
                    loss = self.loss_fn(y_pred.squeeze(-1), y_batch)
                    n_true_b, n_samples_b = multilabel_accuracy(y_pred.squeeze(-1), y_batch, threshold) 
                    n_true_val += n_true_b
                    n_samples_val += n_samples_b
                    acc_loss_val += loss.item()

            acc = n_true/n_samples
            acc_val = n_true_val/n_samples_val
            
            print(f"Train epoch {epoch+1}: Train -> [Loss: {round(acc_loss/len(self.train), 3)}\tAcc: {round(acc*100, 2)}%], Valid -> [Loss: {round(acc_loss_val/len(self.valid), 3)}\tAcc: {round(acc_val*100, 2)}%]\n")
            self.losses_train.append(acc_loss/len(self.train))
            self.losses_valid.append(acc_loss_val/len(self.valid))

    def plot(self):
        plt.plot(learner.losses_train)
        plt.plot(learner.losses_valid)
        plt.ylabel('Error')
        plt.xlabel('Epochs')
        plt.legend(["Entrenamiento", "Validación"])
        plt.show()

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_fn = torch.nn.BCEWithLogitsLoss()

In [ ]:
if size == 128: 
    bs=256
else: 
    bs = 64

train_dataloader = DataLoader(train_dataset, batch_size=bs, shuffle=True, num_workers=0)
valid_dataloader = DataLoader(valid_dataset, batch_size=bs, shuffle=True, num_workers=0)

In [ ]:
learner = Learner(model, train_dataloader, valid_dataloader, optimizer, loss_fn)

En ocasiones, al ejecutar la celda de abajo aparece un error relacionado con "ndim". Sin conocer el motivo detrás de él, se soluciona volviendo a cargar los datos en RAM.

In [ ]:
learner.train=train_dataloader
learner.valid=valid_dataloader
learner.fit(3, 0.0003, 0.1, 0.5)

Train epoch 1: [0/38	Loss: 0.145	Acc: 94.92%]
Train epoch 1: Train -> [Loss: 0.15	Acc: 93.94%], Valid -> [Loss: 0.237	Acc: 89.96%]

Train epoch 2: [0/38	Loss: 0.102	Acc: 96.09%]
Train epoch 2: Train -> [Loss: 0.125	Acc: 94.87%], Valid -> [Loss: 0.205	Acc: 92.15%]

Train epoch 3: [0/38	Loss: 0.089	Acc: 97.66%]
Train epoch 3: Train -> [Loss: 0.112	Acc: 95.56%], Valid -> [Loss: 0.25	Acc: 92.9%]



In [ ]:
learner.plot()

### EfficientNet con Transfer Learning y metadatos 
Similar al apartado anterior, aquí se combina la red preentrenada con los metadatos de la imagen analizada. Para ello se pasan estos metadatos por un percetrón multicapa y la salida se combina con la salida de las capas convolucionales de EfficientNet. Con este nuevo input se volverá a pasar por un perceptrón multicapa que tendrá como output el resultado de la red.

In [ ]:
!pip install efficientnet_pytorch
from efficientnet_pytorch import EfficientNet

class MetaEfficientNet(nn.Module):
    def __init__(self, n_neurons=128):
        super().__init__()
        self.conv = EfficientNet.from_pretrained('efficientnet-b1')
        self.conv._fc = nn.Linear(in_features=1280, out_features=512, bias=True)
        self.MP = nn.Sequential(nn.Linear(12, n_neurons),
                                nn.BatchNorm1d(n_neurons),
                                nn.ReLU(),
                                nn.Dropout(p=0.2)
                                )
        self.out = nn.Linear(512+n_neurons, 1)
    
    def forward(self, x):
        image, meta = x
        output_image = self.conv(image)
        output_meta = self.MP(meta)
        output = self.out(torch.cat((output_image, output_meta), dim=1))
        return output

net = MetaEfficientNet(256)
model = net.cuda()

Loaded pretrained weights for efficientnet-b1


In [ ]:
model

MetaEfficientNet(
  (conv): EfficientNet(
    (_conv_stem): Conv2dStaticSamePadding(
      3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False
      (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
    )
    (_bn0): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
    (_blocks): ModuleList(
      (0): MBConvBlock(
        (_depthwise_conv): Conv2dStaticSamePadding(
          32, 32, kernel_size=(3, 3), stride=[1, 1], groups=32, bias=False
          (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
        )
        (_bn1): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
        (_se_reduce): Conv2dStaticSamePadding(
          32, 8, kernel_size=(1, 1), stride=(1, 1)
          (static_padding): Identity()
        )
        (_se_expand): Conv2dStaticSamePadding(
          8, 32, kernel_size=(1, 1), stride=(1, 1)
          (static_padding): Identity()
        )
        

In [ ]:
def accuracy(y_pred, y):
    n_true = torch.sum(y_pred.data.max(1, keepdim=True)[1].squeeze(-1) == y).item()
    n_samples = y.size()[0]
    return n_true, n_samples

def multilabel_accuracy(y_pred, y, threshold):
    n_true, n_samples = 0, len(y_pred)
    for pred,target in zip(y_pred, y):
        if len(pred[pred>threshold])>1:
            pred_label = (pred==(pred[pred>threshold].max())).nonzero()
        else:
            pred_label = ((pred>threshold)==True).nonzero()
        label = (target==True).nonzero()
        if str(pred_label)==str(label): n_true += 1
    return n_true, n_samples

class Learner():
    def __init__(self, model, train, valid, optim, loss_fn):
        self.model = model
        self.train = train
        self.valid = valid
        self.optim = optim
        self.loss_fn = loss_fn
        self.lr = 0
        self.losses_train = []
        self.losses_valid = []

    def fit(self, epochs, lr, max_lr, threshold):
        #scheduler = torch.optim.lr_scheduler.OneCycleLR(self.optim, max_lr=max_lr, steps_per_epoch=len(self.train), epochs=epochs)
        if lr != self.lr:
            self.lr == lr
            for param_group in optimizer.param_groups:
                param_group['lr'] = lr
        
        for epoch in range(epochs):
            acc_loss = 0
            n_true = 0
            n_samples = 0
            acc = 0
            
            acc_loss_val = 0
            n_true_val = 0
            n_samples_val = 0

            # Training
            self.model = self.model.train()
            for count, sample in enumerate(self.train):
                data, meta = sample
                X_batch = data["image"].to("cuda:0")
                y_batch = data["label"].float().to("cuda:0")
                y_pred = self.model([X_batch, meta.to("cuda:0")])
                loss = self.loss_fn(y_pred.squeeze(-1), y_batch)
                n_true_b, n_samples_b = multilabel_accuracy(y_pred.squeeze(-1), y_batch, threshold) 
                n_true += n_true_b
                n_samples += n_samples_b
                if count % 50 == 0:
                    acc = n_true/n_samples
                    print(f"Train epoch {epoch+1}: [{count}/{len(self.train)}\tLoss: {round(loss.item(), 3)}\tAcc: {round(acc*100, 2)}%]")
                acc_loss += loss.item()

                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                #scheduler.step()

            # Validation
            self.model = self.model.eval()
            i=0
            with torch.no_grad():
                for sample_val in self.valid:
                    data, meta = sample_val
                    X_batch = data["image"].to("cuda:0")
                    y_batch = data["label"].float().to("cuda:0")
                    y_pred = self.model([X_batch, meta.to("cuda:0")])
                    loss = self.loss_fn(y_pred.squeeze(-1), y_batch)
                    n_true_b, n_samples_b = multilabel_accuracy(y_pred.squeeze(-1), y_batch, threshold) 
                    n_true_val += n_true_b
                    n_samples_val += n_samples_b
                    acc_loss_val += loss.item()

            acc = n_true/n_samples
            acc_val = n_true_val/n_samples_val
            
            print(f"Train epoch {epoch+1}: Train -> [Loss: {round(acc_loss/len(self.train), 3)}\tAcc: {round(acc*100, 2)}%], Valid -> [Loss: {round(acc_loss_val/len(self.valid), 3)}\tAcc: {round(acc_val*100, 2)}%]\n")
            self.losses_train.append(acc_loss/len(self.train))
            self.losses_valid.append(acc_loss_val/len(self.valid))

    def plot():
        plt.plot(learner.losses_train)
        plt.plot(learner.losses_valid)
        plt.ylabel('Error')
        plt.xlabel('Epochs')
        plt.legend(["Entrenamiento", "Validación"])
        plt.show()

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_fn = torch.nn.BCEWithLogitsLoss()

In [ ]:
if size == 128: 
    bs=256
else: 
    bs = 64

train_dataloader = DataLoader(train_dataset, batch_size=bs, shuffle=True, num_workers=0)
valid_dataloader = DataLoader(valid_dataset, batch_size=bs, shuffle=True, num_workers=0)

In [ ]:
learner = Learner(model, train_dataloader, valid_dataloader, optimizer, loss_fn)

En ocasiones, al ejecutar la celda de abajo aparece un error relacionado con "ndim". Sin conocer el motivo detrás de él, se soluciona volviendo a cargar los datos en RAM.

In [ ]:
learner.train=train_dataloader
learner.valid=valid_dataloader
learner.fit(3, 0.0003, 0.1, 0.5)

In [ ]:
learner.plot()